# Building Graph-based Applications

This notebook shows a way to use `ipyradiant` widgets to build a simple tab-based
application.

In [ ]:
import ipywidgets as W
import traitlets as T
from rdflib import BNode, Graph

from ipyradiant import CytoscapeVisualizer, FileManager, PathLoader, QueryWidget

## The App Class

This class-based widget captures both the overall layout, as well its individual
components. Values are
[linked](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#Linking-Widgets)
to create new behaviors.

In [ ]:
class RadiantTabs(W.Tab):
    graph = T.Instance(Graph, allow_none=True)
    file_manager = T.Instance(FileManager)
    query = T.Instance(QueryWidget)
    log = W.Output()

    def __init__(self, graph: Graph = None, *args, **kwargs):
        if graph is not None:
            kwargs["graph"] = graph
        if "layout" not in kwargs:
            kwargs["layout"] = {
                "min_height": "400px",
                "flex": "1",
            }
        super().__init__(*args, **kwargs)
        T.link((self.file_manager, "graph"), (self, "graph"))
        T.link((self, "graph"), (self.query, "graph"))

        self.children = [self.file_manager, self.query]
        self.set_title(0, "Load")
        self.set_title(1, "Query")

    @T.default("graph")
    def make_default_graph(self):
        return Graph()

    @T.default("file_manager")
    def make_default_load_widget(self):
        return FileManager(loader=PathLoader(path="data"))

    @T.default("query")
    def make_default_query_widget(self):
        qw = QueryWidget()
        # set default query
        qw.query = """SELECT DISTINCT ?s ?p ?o\nWHERE {\n    ?s ?p ?o .\n}\nLIMIT 10"""
        return qw

## Showing the App

In [ ]:
tabs = RadiantTabs()
tabs